In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/Retail Pulse ML Assignment Data"

/content/drive/My Drive/Retail Pulse ML Assignment Data


In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join, isdir
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
import numpy as np
from numpy import savez_compressed
from numpy import load
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,UpSampling2D
from keras.layers import Input
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import InputLayer, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications import InceptionV3
# from tensorflow.keras.applications import Xception # TensorFlow ONLY
# from tensorflow.keras.applications import VGG16
# from tensorflow.keras.applications import VGG19
# from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from matplotlib import pyplot as plt
%matplotlib inline
from keras.regularizers import l2

In [ ]:
id = []
family = []

In [ ]:
f = open('images_family_train.txt','r')
for line in f:
  idd = line[:7]
  fam = line[7:]
  fam = fam.strip("\n")
  fam = fam.strip(" ")
  family.append(fam)
  id.append(idd)

In [ ]:
variants = []

In [ ]:
g = open("images_variant_train.txt","r")
for line in g:
  var = line[7:]
  var = var.strip("\n")
  var = var.strip(" ")
  variants.append(var)


In [ ]:
data = pd.DataFrame()
# data['Id'] = id
data['Variants'] = variants
data['Family'] = family

In [ ]:
new_data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
new_data

,Variants,Family
0,707-320,Boeing 707
1,727-200,Boeing 727
2,737-200,Boeing 737
3,737-300,Boeing 737
4,737-400,Boeing 737
...,...,...
95,Spitfire,Spitfire
96,Tornado,Tornado
97,Tu-134,Tu-134
98,Tu-154,Tu-154


In [ ]:
data_with_id = pd.DataFrame()
data_with_id['Id'] = id
data_with_id['Variants'] = variants
data_with_id['Family'] = family

In [ ]:
ytrain = pd.DataFrame()
ytrain['family'] = family

In [ ]:
# generate binary values using get_dummies
fam_df = pd.get_dummies(ytrain, columns=["family"], prefix=["Type_is"] )
# merge with main df bridge_df on key values
ytrain = ytrain.join(fam_df)
ytrain.drop("family",axis=1, inplace=True)

In [ ]:
val_family = []
f = open('images_family_val.txt','r')
for line in f:
  var = line[7:]
  var = var.strip("\n")
  var = var.strip(" ")
  val_family.append(var)
ytest = pd.DataFrame()
ytest['val_family'] = val_family
variants_val_df = pd.get_dummies(ytest, columns=["val_family"], prefix=["Type_is"] )
ytest = ytest.join(variants_val_df)
ytest.drop("val_family",axis=1, inplace=True)

In [ ]:
train_path = "/content/drive/My Drive/Retail Pulse ML Assignment Data/train"
val_path = "/content/drive/My Drive/Retail Pulse ML Assignment Data/val"

In [ ]:
training_set = []
i = 0
for f in sorted(listdir(train_path)):
  all_imgs_in_folder = []
  for g in sorted(listdir(join(train_path,f))):
    img = load_img(join(join(train_path,f),g),target_size=(32,32))
    img = np.array(img, dtype=np.float16) / 255.0
    all_imgs_in_folder.append(img)
    print(i)
    i+=1
  training_set.extend(all_imgs_in_folder)

In [ ]:
savez_compressed("training_set_array_32.npz",training_set)

In [ ]:
dict_data = load('training_set_array_32.npz')
training_set = dict_data['arr_0']

In [ ]:
# training_set = training_set.reshape(-1,28,28,1)

In [ ]:
training_set = training_set.reshape(-1,32,32,3)

In [ ]:
val_set = []
i = 0
for f in sorted(listdir(val_path)):
  all_imgs_in_folder = []
  for g in sorted(listdir(join(val_path,f))):
    img = load_img(join(join(val_path,f),g),target_size=(32,32))
    img = np.array(img, dtype=np.float16) / 255.0
    all_imgs_in_folder.append(img)
    print(i)
    i+=1
  val_set.extend(all_imgs_in_folder)

In [ ]:
savez_compressed("val_set_array_32.npz",val_set)

In [ ]:
dict_data = load('val_set_array_32.npz')
val_set = dict_data['arr_0']

In [ ]:
# val_set = val_set.reshape(-1,28,28,1)

In [ ]:
val_set = val_set.reshape(-1,32,32,3)

In [ ]:
datagen = ImageDataGenerator(rotation_range=12,
                             width_shift_range= 1.5,
                             height_shift_range = 0.4,
                             horizontal_flip = True)

In [ ]:
datagen.fit(training_set)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3),kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(70, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_33 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
batch_normalization_34 (Batc (None, 13, 13, 32)        128       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 4, 4, 64)         

In [ ]:
filepath="/content/drive/My Drive/Retail Pulse ML Assignment Data/weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(datagen.flow(training_set, ytrain, batch_size=32),
          steps_per_epoch=(3*len(training_set) / 32), epochs=100,validation_data=(val_set,ytest),callbacks=callbacks_list)

Epoch 1/100
313/312 [==============================] - 11s 34ms/step - loss: 4.2153 - accuracy: 0.0712 - val_loss: 4.1058 - val_accuracy: 0.0813

Epoch 00001: val_accuracy improved from -inf to 0.08131, saving model to /content/drive/My Drive/Retail Pulse ML Assignment Data/weights_best.hdf5
Epoch 2/100
313/312 [==============================] - 11s 34ms/step - loss: 4.1420 - accuracy: 0.0740 - val_loss: 4.0326 - val_accuracy: 0.0849

Epoch 00002: val_accuracy improved from 0.08131 to 0.08491, saving model to /content/drive/My Drive/Retail Pulse ML Assignment Data/weights_best.hdf5
Epoch 3/100
313/312 [==============================] - 11s 34ms/step - loss: 4.0558 - accuracy: 0.0780 - val_loss: 4.6757 - val_accuracy: 0.0792

Epoch 00003: val_accuracy did not improve from 0.08491
Epoch 4/100
313/312 [==============================] - 11s 34ms/step - loss: 4.0079 - accuracy: 0.0845 - val_loss: 3.9563 - val_accuracy: 0.0972

Epoch 00004: val_accuracy improved from 0.08491 to 0.09721, savi

KeyboardInterrupt: ignored

In [ ]:
# train_generator = train_datagen.flow_from_directory(
#     directory=train_path,
#     target_size=(28, 28),
#     color_mode="grayscale",
#     batch_size=32,
#     class_mode="categorical",
#     shuffle=False
# )

Found 3334 images belonging to 100 classes.


In [ ]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=40,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest')

In [ ]:
# val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# valid_generator= val_datagen.flow_from_directory(
#     directory=val_path,
#     target_size = (28,28),
#     color_mode = "grayscale",
#     batch_size =32,
#     class_mode="categorical"
# )

Found 3333 images belonging to 100 classes.


In [ ]:
# cnn4.fit_generator(
#         train_generator,
#         steps_per_epoch=3334 // 32,
#         epochs=10
#         )

In [ ]:
# base_model = ResNet50(weights= "imagenet", include_top=False, input_shape= (32,32,3))
# for layer in base_model.layers:
#         layer.trainable = False

# model = Sequential()
# model.add(Flatten())
# model.add(Dense(200, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
# model.add(Dense(400, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())

# # softmax classifier
# model.add(Dense(100,activation='softmax'))

# pretrainedInput = base_model.input
# pretrainedOutput = base_model.output
# output = model(pretrainedOutput)
# model = Model(pretrainedInput, output)        

In [ ]:
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.7)(x)
# predictions = Dense(100, activation= 'softmax')(x)
# model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
# rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     shear_range=0.2,
#     zoom_range=0.2

In [ ]:
input_img = Input(shape = (28, 28, 1))
def encoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    return conv4

def decoder(conv4):    
    #decoder
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4) #7 x 7 x 128
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded

In [ ]:
model = Model(input_img, decoder(encoder(input_img)))
model.compile(loss='mean_squared_error', optimizer = keras.optimizers.RMSprop())